LSTM MODEL
Yufan qian

In [13]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import nltk
import math

In [14]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [16]:
#adapted from Homework 3
def train_lstm_classification(model, train_dataset, valid_dataset, epochs=10, batch_size=32, learning_rate=.001, print_frequency=25):

    criteria = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


    epochs = epochs
    batch_size = batch_size
    print_frequency = print_frequency

    
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=128, shuffle=False)

    print('Total train batches: {}'.format(train_dataset.__len__() / batch_size))

    best_accuracy = 0.0
    best_model_sd = None

    for i in range(epochs):
        print('### Epoch: ' + str(i+1) + ' ###')
    
        model.train()

        avg_loss = 0

        for step, data in enumerate(train_dataloader):

            (x, x_lengths), y = data

            optimizer.zero_grad()

            model_output = model(x, x_lengths)

            loss = criteria(model_output.squeeze(1), y.float())

            loss.backward()
            optimizer.step()

            avg_loss += loss.item()

            if step % print_frequency == (print_frequency - 1):
                print('epoch: {} batch: {} loss: {}'.format(
                    i,
                    step,
                    avg_loss / print_frequency
                ))
                avg_loss = 0

        print('Evaluating...')
        model.eval()
        with torch.no_grad():
            acc = predict(model, valid_dataloader)
            if acc > best_accuracy:
                best_model_sd = copy.deepcopy(model.state_dict())
                best_accuracy = acc

    return model.state_dict(), best_model_sd

In [17]:
class LSTM(nn.Module):
 
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,batch_first=True)
        self.output_layer = nn.Linear(hidden_size, output_size)

        #adapted from Homework 3
    def forward(self, input_batch, input_lengths):
        print('Input batch shape: {}'.format(input_batch.shape))
        embedded_input = self.embedding(input_batch)
        
        print('Embedded input shape: {}'.format(embedded_input.shape))
        packed_input = pack_padded_sequence(embedded_input, input_lengths, batch_first=True, enforce_sorted=False)
        
        packed_output, (hn, cn) = self.lstm(packed_input)
        
        hn_view = hn.view(self.lstm.num_layers, self.num_directions, input_batch.shape[0], self.lstm.hidden_size)               # Reshape hn for clarity -- first dimension now represents each layer (total set by num_lstm_layers)
        print('hn_view input shape: {}'.format(hn_view.shape))
        
        hn_view_last_layer = hn_view[-1]                                                                                        # Taking the last layer for our final LSTM output
        print('hn_view_last_layer input shape: {}'.format(hn_view_last_layer.shape))
        
        hn_cat = torch.cat([hn_view_last_layer[-2, :, :], hn_view_last_layer[-1, :, :]], dim=1)                                 # Each layer has two directions. We want to use both of these vectors, so concatenate them
        print('hn_cat input shape: {}'.format(hn_cat.shape))
        
        hid = self.relu(self.hidden_1(hn_cat))
        print('hid input shape: {}'.format(hid.shape))
        
        output = self.hidden_2(hid)
        print('output input shape: {}'.format(output.shape))
    
        return output
model = LSTM(input_size, hidden_size, output_size, num_layers)
model, best_model = train_lstm_classification(model, train_dataset, valid_dataset, batch_size=128, epochs=1)

NameError: name 'input_size' is not defined

In [ ]:
model = LSTM(input_size, hidden_size, output_size, num_layers)
model, best_model = train_lstm_classification(model, train_dataset, valid_dataset, batch_size=128, epochs=1)